In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import anndata as ad

from evaluate import evaluate
from prediction import ADTPredictor

In [ ]:
dataset_path = "datasets/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_"
gex_train = ad.read_h5ad(dataset_path + "train_mod1.h5ad")
gex_test = ad.read_h5ad(dataset_path + "test_mod1.h5ad")
adt_train = ad.read_h5ad(dataset_path + "train_mod2.h5ad")
adt_test = ad.read_h5ad(dataset_path + "test_mod2.h5ad")

In [ ]:
# using high-level interface
pipe = ADTPredictor(do_log1p=False)
pipe.fit(gex_train.X.toarray(), adt_train.X.toarray(), gex_test.X.toarray())
adt_pred = pipe.predict(gex_test.X.toarray())
evaluate(adt_pred, adt_test.X.toarray())